# Fully Convolutional Neural Networks

Example of a fully convolutional network from the [paper](http://cvlab.postech.ac.kr/research/deconvnet/)
![Fully convolution net](../images/fcn.png)

## 2.1. Models

In [1]:
import torch as tc
import torch.nn as nn
import numpy as np
import pylab
%pylab inline

from skimage import io, transform, exposure

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/Users/tramx/Library/Python/3.8/lib/python/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['pylab']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
import torchvision as tv

### Building a Simple Segmentation Network

In [3]:
class SimpleSegNet(nn.Module):
    def __init__(self, n_inputs, n_classes=1,  dropout_prob=0.3):
        super(SimpleSegNet, self).__init__()

        self.pipe = nn.Sequential(
            # Encoder
            nn.Conv2d(n_inputs, 8, (3,3), stride=1, padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 16, (3,3), stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, (3,3), stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 32, (3,3), stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, (3,3), stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            
            # Decoder
            nn.Upsample(scale_factor=2),
            nn.Conv2d(32, 16, (3,3), stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(16, 8, (3,3), stride=1, padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 1, (3,3), stride=1, padding=1),
            nn.BatchNorm2d(1),
            nn.Softmax(dim=-1) if n_classes > 1 else nn.Sigmoid()
        )
    def forward(self, x):
        return self.pipe(x)



In [4]:
simple_seg_net = SimpleSegNet(n_inputs=3, n_classes=1)

In [5]:
simple_seg_net.forward(tc.randn(12, 3, 128, 128)).shape

torch.Size([12, 1, 128, 128])

### Loading a Pretrained Model

In [6]:
deeplab = tv.models.segmentation.deeplabv3_resnet50(pretrained=False, num_classes=1)

In [7]:
deeplab.forward(tc.randn(12, 3, 256, 256))["out"].shape

torch.Size([12, 1, 256, 256])

In [8]:
# Loss function definition
# CrossEntropy since we have a Softmax as last layer
criterion = nn.CrossEntropyLoss()

# VOC Example

# Retina Example

In [6]:
ROOT_FOLDER = "./DRIVE/"
task = "training" # "test"

In [7]:
import os

In [40]:
class DriveDataset(tc.utils.data.Dataset):
    """Blood vessel segmentation dataset."""

    def __init__(self, task, root_dir, transform=None):
        """
        Args:
            task (string): current task, either "training" or "test".
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self._build_list_images(task)
    
    def _build_list_images(self, task):
        self.im_paths = []
        self.gt_paths = []
        for i in range(21, 41):
            self.im_paths.append(os.path.join(task, "images", "{0}_training.tif".format(i)))
            self.gt_paths.append(os.path.join(task, "1st_manual", "{0}_manual1.gif".format(i)))

    def __len__(self):
        return len(self.im_paths)

    def __getitem__(self, idx):
        if tc.is_tensor(idx):
            idx = idx.tolist()

        im_name = os.path.join(self.root_dir, self.im_paths[idx])
        gt_name = os.path.join(self.root_dir, self.gt_paths[idx])
        image = io.imread(im_name)
        label = io.imread(gt_name)
        print(np.unique(image), np.unique(label))

        if self.transform:
            image = self.transform(image)
            label = self.transform(label)
        return image, label

In [41]:
transforms = tv.transforms.Compose([tv.transforms.ToTensor(),
                              #tv.transforms.Normalize(0.5, 0.5),
                              tv.transforms.RandomCrop((256, 256))])

In [42]:
retina_dataset = DriveDataset(task, ROOT_FOLDER, transform=transforms)

In [43]:
retina_dataset[0]

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

(tensor([[[1.0000, 1.0000, 1.0000,  ..., 0.8745, 0.8667, 0.8510],
          [1.0000, 1.0000, 1.0000,  ..., 0.8510, 0.8510, 0.8275],
          [1.0000, 1.0000, 1.0000,  ..., 0.8392, 0.8392, 0.8196],
          ...,
          [0.1961, 0.2784, 0.4549,  ..., 0.7686, 0.7843, 0.7922],
          [0.1137, 0.1569, 0.2510,  ..., 0.7647, 0.7765, 0.7882],
          [0.0275, 0.0824, 0.1725,  ..., 0.7804, 0.7882, 0.7922]],
 
         [[0.9569, 0.9373, 0.9412,  ..., 0.4549, 0.4510, 0.4353],
          [0.9569, 0.9608, 0.9686,  ..., 0.4353, 0.4510, 0.4275],
          [0.9490, 0.9569, 0.9608,  ..., 0.4196, 0.4392, 0.4196],
          ...,
          [0.0000, 0.0000, 0.0588,  ..., 0.4235, 0.4353, 0.4431],
          [0.0118, 0.0000, 0.0000,  ..., 0.4118, 0.4118, 0.4235],
          [0.0353, 0.0078, 0.0000,  ..., 0.4196, 0.4275, 0.4314]],
 
         [[0.5294, 0.5098, 0.5098,  ..., 0.2902, 0.2941, 0.2824],
          [0.5255, 0.5333, 0.5373,  ..., 0.2824, 0.2941, 0.2784],
          [0.5176, 0.5255, 0.5294,  ...,

In [36]:
dataloader = tc.utils.data.DataLoader(retina_dataset, batch_size=4,
                        shuffle=True, num_workers=0)


In [37]:
simple_seg_net = SimpleSegNet(n_inputs=3, n_classes=1)

In [38]:
# Loss function definition
# CrossEntropy since we have a Softmax as last layer
criterion = nn.CrossEntropyLoss()
# Optimizers takes as input parameters to optimize and a learning rate
optimizer = tc.optim.SGD(simple_seg_net.parameters(), lr=0.5)
epochs = 100 # overall number of iterations

In [39]:

losses = []
for e in range(epochs):
    
    for i_batch, sample_batched in enumerate(dataloader):
        images, labels = sample_batched # getting the images and their labels

        optimizer.zero_grad() # Zero-out the gradients, cleaning the optimizer

        output = simple_seg_net.forward(images) # we apply the model on the batch images

        loss = criterion(output, labels) # we compute the loss between the model's outputs and the labels

        loss.backward() # we backpropagate the loss in the model to compute all the gradiants

        optimizer.step() # final we update the weights

        print("Epoch {}/{} - loss {}".format(e+1, epochs, loss.detach().numpy()))
        losses.append(loss.detach().numpy())
plt.plot(range(len(losses)), losses)

Epoch 1/100 - loss -0.0
Epoch 1/100 - loss -0.0
Epoch 1/100 - loss -0.0
Epoch 1/100 - loss -0.0
Epoch 1/100 - loss -0.0
Epoch 2/100 - loss -0.0
Epoch 2/100 - loss -0.0
Epoch 2/100 - loss -0.0
Epoch 2/100 - loss -0.0
Epoch 2/100 - loss -0.0
Epoch 3/100 - loss -0.0
Epoch 3/100 - loss -0.0
Epoch 3/100 - loss -0.0
Epoch 3/100 - loss -0.0
Epoch 3/100 - loss -0.0
Epoch 4/100 - loss -0.0
Epoch 4/100 - loss -0.0
Epoch 4/100 - loss -0.0


KeyboardInterrupt: 